In [1]:
# upload audio file
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [10]:
import shutil
shutil.copy("/content/drive/MyDrive/Audio/test.wav","/content/")

'/content/test.wav'

In [3]:
!apt-get update -qq
!apt-get install -y locales
!locale-gen en_US.UTF-8

Reading package lists... Done
Building dependency tree       
Reading state information... Done
locales is already the newest version (2.31-0ubuntu9.9).
0 upgraded, 0 newly installed, 0 to remove and 24 not upgraded.
Generating locales (this might take a while)...
  en_US.UTF-8... done
Generation complete.


In [4]:
import os
os.environ['LC_ALL'] = 'en_US.UTF-8'
os.environ['LANG'] = 'en_US.UTF-8'

In [5]:
!pip install -q git+https://github.com/openai/whisper.git > /dev/null
!pip install -q git+https://github.com/pyannote/pyannote-audio > /dev/null
!pip install -q spacy
!pip install -q sentencepiece
!pip install -q bert-extractive-summarizer

import subprocess 
from tqdm import tqdm 
import whisper 

import torch 

from sklearn.cluster import AgglomerativeClustering

import json
import numpy as np 
import pandas as pd
from summarizer import Summarizer 
from transformers import AutoModelForSeq2SeqLM,AutoTokenizer 
import datetime,time
 
import pyannote.audio 
from pyannote.audio.pipelines.speaker_verification import PretrainedSpeakerEmbedding

from pyannote.audio import Audio 
from pyannote.core import Segment 

import wave 
import contextlib 

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 57.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 98.2 MB/s eta 0:00:00


In [11]:
# Set input parameters
path = "/content/drive/MyDrive/Audio/test.wav"
model_size = "medium"
language = "English"

# Split filename and remove file extension
filename = path.split("/")[-1][:-3].split(' ')

# Set model name based on language and model size
model_name = model_size
if language == 'English' and model_size != 'medium':
    model_name += '.en'

# Convert input file to WAV format if not already in WAV format
if path[-3:] != 'wav':
    subprocess.call(['ffmpeg', '-i', path, 'audio.wav', '-y'])
    path = 'audio.wav'

In [12]:
"""
# define model parameters and load model
# Transformer based Seq2Seq model trained on weak supervision
"""

# Load the Whisper model
model = whisper.load_model(model_size)

100%|█████████████████████████████████████| 1.42G/1.42G [01:12<00:00, 21.0MiB/s]


In [13]:
"""
    Performs speaker verification using cosine distance between speaker embeddings
    
    # dict_keys(['text', 'segments', 'language'])
    id : id of audio chunk/segment 
    start & end : start & end time of audio chunk
    text : transcribed text
    tokens : generated token list
    temperature : non-deterministic factor (0 means deterministic model)
    avg_logprob : uses additive margin softmax loss 
    compression_ratio : .WAV is windows compressed but .wav is uncompressed
    no_speech_prob : probability of silence detection
"""

# Transcribe the audio using the Whisper model
result = model.transcribe(path)
segments = result["segments"]

In [14]:
# Open output file for writing the transcription
f = open(f"{filename[0] if len(filename) > 0 else 'default'}_{filename[1] if len(filename) > 1 else 'transcript'}_whisper_transcript.txt", "w")
text = ""

# Iterate through segments and write the transcribed text to the output file
for (i, segment) in tqdm(enumerate(segments)):
    try:
        if "text" in segment and len(segment["text"]) > 1:
            f.write(segment["text"][1:] + ' ')
    except Exception as e:
        print(f"Error at segment {i}: {segment}")
        print(f"Error details: {e}")
        continue

18it [00:00, 106785.67it/s]
